# Setup

In [2]:
import pandas_datareader as pdr
import datetime
import numpy as np
import pandas as pd
import os

In [3]:
os.getcwd()

'/Users/josiahdavis/Documents/Berkeley/STAT222/stocks222/sandbox/josiah'

# Define functions

In [4]:
def get_metadata():    
    # Read in the new york stock exchange metadata
    df = pd.read_csv('../../../data/nyse.csv')
    print('Number of stock tickers: ' + str(df.shape[0]))
    print('Number of Sectors: ' + str(df.Sector.unique().shape[0]))
    print('Number of Industries: ' + str(df.industry.unique().shape[0]))
    print(df.Sector.value_counts())
    print(df.iloc[:10,:6])
    return(df)

def get_daily_data(tickers):
    start = datetime.datetime(1986, 1, 1)
    end = datetime.datetime(2016, 12, 31)
    df = pdr.get_data_yahoo(tickers, start, end)
    return(df.Close)

def to_monthly(df):
    # ffill: gets the previous value if there is no value for the business month end
    # e.g., the 29th is the last business day of 3/1991 but the 28th is last day with data
    # BM: Business Month
    # x.iloc[-1]: returns the last value
    df = df.asfreq('BM', method = 'ffill')
    return(df)

def get_returns(df):
    # Ignore the fact that share prices minorly during off hours for now
    df = df.apply(lambda x: (x - x.shift(1)) / x.shift(1))
    return(df)

def remove_missing(df):
    # Remove stocks without complete records
    n_original = str(df.shape[1])    
    # First row will always have NaN value
    df = df.iloc[1:,].dropna(axis = 1, how = 'any')
    n_complete = str(df.shape[1])
    print('Original stocks: ', n_original,'Complete records: ', n_complete)
    return(df)

# Get the data

In [5]:
nyse = get_metadata()

Number of stock tickers: 3155
Number of Sectors: 13
Number of Industries: 132
n/a                      975
Consumer Services        449
Finance                  354
Energy                   225
Public Utilities         209
Basic Industries         205
Capital Goods            184
Technology               160
Consumer Non-Durables    116
Health Care              107
Transportation            62
Consumer Durables         58
Miscellaneous             51
Name: Sector, dtype: int64
  Symbol                    Name LastSale MarketCap IPOyear             Sector
0    DDD  3D Systems Corporation    14.17     $1.6B     n/a         Technology
1    MMM              3M Company   191.51  $114.45B     n/a        Health Care
2   WBAI         500.com Limited    12.77  $529.93M    2013  Consumer Services
3   WUBA             58.com Inc.    37.31     $5.4B    2013         Technology
4    AHC   A.H. Belo Corporation      6.1  $132.23M     n/a  Consumer Services
5   ATEN      A10 Networks, Inc.     9.18  $

In [23]:
# Get data from Yahoo! finance
daily = get_daily_data(nyse.loc[:,'Symbol'].tolist())

# Read data already got from Yahoo! finance
# daily = pd.read_csv('../../../data/daily.csv', parse_dates = True, index_col = 0)

In [24]:
daily.head()

,A,AA,AAC,AAN,AAP,AAT,AAV,AB,ABB,ABBV,...,ZB^H,ZEN,ZF,ZNH,ZOES,ZPIN,ZTO,ZTR,ZTS,ZX
Date,,,,,,,,,,,,,,,,,,,,,
1986-01-02,NaN,NaN,NaN,17.000011,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-01-03,NaN,NaN,NaN,16.749989,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-01-06,NaN,NaN,NaN,16.749989,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-01-07,NaN,NaN,NaN,16.749989,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-01-08,NaN,NaN,NaN,17.250031,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
daily.to_csv('../../../data/daily.csv')

In [26]:
# Format data into monthly returns (dropping a couple of stocks with weird days)
monthly = to_monthly(daily.drop(['CBX', 'SAB', 'EAI', 'EMP', 'ELC'], axis = 1).dropna(how = 'all'))
monthly = get_returns(monthly)
monthly = remove_missing(monthly)

Original stocks:  3150 Complete records:  313


In [29]:
monthly.head()

,AAN,ABM,ABT,ABX,ADM,ADX,AEG,AEM,AEP,AET,...,WRB,WSM,WSO,WTR,WWW,WY,XEL,XOM,XRX,Y
Date,,,,,,,,,,,,,,,,,,,,,
1986-02-28,0.059698,-0.011299,0.027273,-0.086212,0.013575,0.042254,-0.032053,-0.081633,0.039216,0.100671,...,0.192309,0.289474,0.078435,0.035294,-0.100006,0.122606,0.097222,0.009662,0.099222,0.087267
1986-03-31,-0.014078,0.120000,0.221239,0.075472,0.062500,0.047298,0.013246,0.022222,0.047170,0.054878,...,0.168008,0.071429,-0.036365,-0.034091,0.099995,0.051195,0.021097,0.066986,-0.049557,0.053947
1986-04-30,0.071426,0.096939,0.000000,0.385978,-0.109244,0.012903,0.081699,-0.065217,-0.067568,-0.065511,...,-0.128883,-0.123810,0.320756,-0.035294,-0.060603,-0.045455,0.024793,0.015695,-0.106145,-0.039950
1986-05-30,-0.079999,-0.046512,0.072464,0.000000,0.070755,0.031847,0.018128,-0.085271,0.000000,0.049485,...,0.211362,0.043478,0.057141,0.018293,-0.021504,0.010204,0.020161,0.057395,0.020833,-0.009103
1986-06-30,0.072462,0.000000,-0.418919,0.037970,-0.356828,0.000000,0.005935,0.059322,0.028986,-0.047151,...,-0.302073,-0.260417,-0.081081,0.005988,-0.054942,-0.060606,-0.446640,0.016701,-0.083673,0.032808


In [30]:
monthly.to_csv('../../../data/monthly.csv')